In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

#import data 
data=pd.read_excel("games.xlsx")


In [ ]:
#if the number of moves<10, it's drop in database
data=data.drop(data.loc[data['turns']<=10].index)
data=data.reset_index(drop=True)

In [ ]:
#missing data check
data.isnull().sum()

In [ ]:
#Missing Values 
#IterativeImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
testmissing = data.iloc[:9000,2:4]
trainmissing = data.iloc[9000:,2:4]
imp = IterativeImputer(random_state=0)
imp.fit(trainmissing)
data.iloc[:9000,2:4] = np.round(imp.transform(testmissing))

In [ ]:
#Label Encoder and OneHotEncoder
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

#rated
data.iloc[:,1] = le.fit_transform(data.iloc[:,1])

#victory status
ohe = preprocessing.OneHotEncoder()
vicstatus = data.iloc[:,4].to_numpy()
vicstatus = vicstatus.reshape(-1,1)
ohe_vicstatus = ohe.fit_transform(vicstatus).toarray()
dfvicstatus=pd.DataFrame(data=ohe_vicstatus,columns=["draw","mate","outoftime","resign"])

#opening_eco
eco = data['opening_eco']
for i in range(len(eco)):  
    eco[i]=eco[i].replace("A","1")
    eco[i]=eco[i].replace("B","2")
    eco[i]=eco[i].replace("C","3")
    eco[i]=eco[i].replace("D","4")
    eco[i]=eco[i].replace("E","5")

eco=pd.to_numeric(eco)

In [ ]:
#attributes and label dataset
X = pd.concat([data["rated"],data["game_time"],data["turns"],dfvicstatus,data["black_rating"],data["white_rating"]],axis=1)

y = data["winner"]
y = le.fit_transform(y) #Label Encoder applied to label set
classes =["black","draw","white"] # label classes

In [ ]:
#Standard Scaler
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_sc = sc.fit_transform(X)

In [ ]:
#KFold uygulaması
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.metrics import confusion_matrix , ConfusionMatrixDisplay ,classification_report
#function for comparing models
def output(est,X,color):
   
    for model in est:
        accu = cross_val_score(estimator = model, X = X, y=y,cv=5)
        y_pred = cross_val_predict(model,X,y,cv=5)
        conf_matrix = confusion_matrix(y, y_pred)
        disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix,display_labels=["black","draw","white"])
        disp.plot(cmap=color)
        disp.ax_.set_title(model)
        print("-------- ",model," report -------- \n")
        print(classification_report(y,y_pred,target_names=classes))
        print("\n")

In [ ]:
#import some models
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state=0)

from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)

from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

In [ ]:
#Some models applied to Grid Search CV
"""Random Forest Grid_SearchCv """
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

parameters = [{'n_estimators': [50,100,150,200], 'criterion':['gini',"entropy"]}]
grids = GridSearchCV(estimator = rfc, param_grid = parameters, scoring = 'accuracy', cv = 5, n_jobs = -1)
grids.fit(X_sc, y)
best_accuracy , best_parameter = grids.best_score_ , grids.best_params_
print('En iyi acc: ', best_accuracy)
print('En iyi acc veren parametreler: ', best_parameter,"\n")
grid_rfc = RandomForestClassifier(n_estimators=150,criterion="entropy")


"""KNN Grid_SearchCv """
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

parameters = [{'n_neighbors': [3,5,7,9,11,13,15,17,19,21], 'weights':['uniform',"distance"],"metric" : ["euclidean","manhattan","minkowski"]}]
grids = GridSearchCV(estimator = knn, param_grid = parameters, scoring = 'accuracy', cv = 5, n_jobs = -1)
grids.fit(X_sc, y)
best_accuracy , best_parameter = grids.best_score_ , grids.best_params_
print('En iyi acc: ', best_accuracy)
print('En iyi acc veren parametreler: ', best_parameter,"\n")
grid_knn = KNeighborsClassifier(n_neighbors=21,weights="distance",metric="manhattan")


"""SVM Grid_SearchCv """
from sklearn.svm import SVC
svm = SVC()

parameters = [{'C': [0.25, 0.5, 0.75, 1], 'kernel':['linear']}, {'C': [0.25, 0.5, 0.75, 1], 'kernel':['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5]}]
grids = GridSearchCV(estimator = svm, param_grid = parameters, scoring = 'accuracy', cv = 5, n_jobs = -1)
grids.fit(X_sc, y)
best_accuracy , best_parameter = grids.best_score_ , grids.best_params_
print('En iyi acc: ', best_accuracy)
print('En iyi acc veren parametreler: ', best_parameter,"\n")
grid_svm = SVC(C=0.75,kernel="linear")


In [ ]:
#printing results
#not parameters on models
est = [logreg,clf,rfc,knn,svm,gnb]
output(est,X_sc,"pink")

In [ ]:
#Three models applied to Grid Search CV 
grid_est = [grid_rfc,grid_knn,grid_svm]
output(grid_est,X_sc,"hot")

In [ ]:
#Linear Discriminant Analysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components = 2)
X_lda = lda.fit_transform(X_sc, y)

In [ ]:
#LDA and Grid Search CV
grid_lda_est = [logreg,clf,grid_rfc,grid_knn,grid_svm,gnb]
output(grid_lda_est,X_lda,"bone")